# DATA CLEANING

## Import packages and explore raw data

In [1]:
#Import initial packages
import pandas as pd

In [2]:
#Load raw dataset
data = pd.read_csv("datasets/raw_data_final/raw_final_dataset.csv")

In [3]:
#Get the dimensions (rows, columns) of the dataframe
data.shape
#The dataset contains 3555 records and 113 records

(3555, 113)

In [4]:
#View the first few rows of the dataframe
data.head()

,CLUB_NAME,PLAYER_NAME,PLAYER_VALUE,PLAYER_HREF,CLUB_HREF,LEAGUE_NAME,LEAGUE_COUNTRY,UrlFBref,CURRENT_INTERNATIONAL,DATE_OF_BIRTH,...,TKL_CHALLENGES,ATT_CHALLENGES,LOST_CHALLENGES,BLOCKS_BLOCKS,SH_BLOCKS,PASS_BLOCKS,INT,TKL+INT,CLR,ERR
0,Manchester City,Ederson,€40.00m,https://www.transfermarkt.com/ederson/profil/s...,https://www.transfermarkt.us/manchester-city/s...,Premier League,England,https://fbref.com/en/players/3bb7b8b4/Ederson,Brazil,1993-08-17,...,1.0,2.0,1.0,0.0,0.0,0.0,1.0,4.0,10.0,2.0
1,Manchester City,S. Ortega,€9.00m,https://www.transfermarkt.com/stefan-ortega/pr...,https://www.transfermarkt.us/manchester-city/s...,Premier League,England,https://fbref.com/en/players/c1242d4e/Stefan-O...,Germany,1992-11-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Manchester City,Z. Steffen,€3.50m,https://www.transfermarkt.com/zack-steffen/pro...,https://www.transfermarkt.us/manchester-city/s...,Premier League,England,https://fbref.com/en/players/42130443/Zack-Ste...,United States,1995-04-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Manchester City,S. Carson,€200k,https://www.transfermarkt.com/scott-carson/pro...,https://www.transfermarkt.us/manchester-city/s...,Premier League,England,https://fbref.com/en/players/3e1550ee/Scott-Ca...,England,1985-09-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Manchester City,R. Dias,€80.00m,https://www.transfermarkt.com/ruben-dias/profi...,https://www.transfermarkt.us/manchester-city/s...,Premier League,England,https://fbref.com/en/players/31c69ef1/Ruben-Dias,Portugal,1997-05-14,...,11.0,18.0,7.0,21.0,13.0,8.0,18.0,39.0,56.0,1.0


In [5]:
#Get general information about the dataframe, like data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3555 entries, 0 to 3554
Columns: 113 entries, CLUB_NAME to ERR
dtypes: float64(96), object(17)
memory usage: 3.1+ MB


## Remove Rows with Incomplete Information

In [6]:
#Remove players who does not have PLAYER_VALUE
data = data.dropna(subset=['PLAYER_VALUE'])

In [7]:
#Remove Players who does not have player statistics
data = data.dropna(subset=['UrlFBref'])

In [8]:
#Remove Players who have MP_Playing as null
#This players does not have statistics as well
data = data.dropna(subset=['MP_PLAYING'])

## Remove unnecesary columns

In [9]:
to_drop = [
    'CURRENT_CLUB', #this club information was after the completion of the 2022-2023 season
    'PLAYER_NAME', #the player ID is going to be the identifier
]
data.drop(columns=to_drop, inplace=True)

## Handle Missing Values

*Handle missing values: Identify columns with missing values and decide how to handle them. You can either drop rows with missing values, impute missing values using techniques like mean, median, or mode.*

In [10]:
#Get missing values and percentage statistics
missing_values = data.isnull().sum()  # Count the missing values in each column
missing_percentage = (data.isnull().sum() / len(data)) * 100  # Calculate the percentage of missing values in each column

# Create a DataFrame to display the missing value information
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Missing Percentage': missing_percentage})
missing_data = missing_data.sort_values(by='Missing Values', ascending=False)

#Save the dataframe in a temp folder
missing_data.to_csv("datasets/temp/TEMP_missing_data.csv")
#The missing_data head shows that the columns with highest percentage of missing values are the goalkeeper statistics, so to simplify the analysis, goalkeeper players (rows) and goalkeeper statistics will be dropped.

In [11]:
#Remove goalkeeper players and goalkeeper stats
goalkeeper_stats_columns = ['GA', 'SOTA', 'SAVES', 'W', 'D', 'L', 'CS', 'PKATT_PENALTY', 'PKA_PENALTY', 'PKSV_PENALTY', 'PKM_PENALTY']
data.drop(columns=goalkeeper_stats_columns, inplace=True)
data = data[data['POSITION'] != 'Goalkeeper']

In [12]:
#Transform "OUTFITTER" column to True if the value is not null
data['OUTFITTER'] = data['OUTFITTER'].apply(lambda x: True if pd.notna(x) else False)
data['OUTFITTER'].value_counts()

False    1457
True      927
Name: OUTFITTER, dtype: int64

In [13]:
#Impute "DIST_STANDARD" column with the median value
median_dist = data['DIST_STANDARD'].median()
data['DIST_STANDARD'] = data['DIST_STANDARD'].fillna(median_dist)

In [14]:
#Transform "PLAYER_AGENT" column to True if the value is not null
data['PLAYER_AGENT'] = data['PLAYER_AGENT'].apply(lambda x: True if pd.notna(x) else False)
data['PLAYER_AGENT'].value_counts()

True     2153
False     231
Name: PLAYER_AGENT, dtype: int64

In [15]:
#Impute "HEIGHT" column with the median value
median_height = data['HEIGHT'].median()
data['HEIGHT'] = data['HEIGHT'].fillna(median_height)

In [16]:
#Impute "FOOT" column with "right" value
foot_values = data['FOOT'].value_counts()
print(foot_values)
#Most of the players are right-footed players, so the null values are imputed with this value ("right")
data['FOOT'] = data['FOOT'].fillna('right')

right    1634
left      624
both       90
Name: FOOT, dtype: int64


In [17]:
#Impute "JOINED" column with the start of the season date
imputed_value = '2022-08-01' #This date is the start date of the 2022-2023 season
data['JOINED'] = data['JOINED'].fillna(imputed_value)

In [18]:
#CREATE "YEAR_BIRTH" column from "DATE_OF_BIRTH" column
#Import datetime library
import datetime as dt

# Extract year from the "DATE_OF_BIRTH" column and create the "YEAR_BIRTH" column
data['YEAR_BIRTH'] = pd.to_datetime(data['DATE_OF_BIRTH']).dt.year

# Impute missing values in "YEAR_BIRTH" using "AGE" column
current_year = 2023
data['YEAR_BIRTH'] = data['YEAR_BIRTH'].fillna(current_year - data['AGE'])
data.drop(columns=['DATE_OF_BIRTH'], inplace=True)

In [19]:
# Function to convert the string values to numeric values in the "PLAYER_VALUE" column
def convert_player_value(value):
    multipliers = {'k': 1000, 'm': 1000000}
    value = value.strip('€')
    currency = value[-1]
    number = value[:-1]
    return float(number) * multipliers[currency]

# Apply the conversion function to the "PLAYER_VALUE" column
data['PLAYER_VALUE'] = data['PLAYER_VALUE'].apply(convert_player_value)

In [20]:
# Impute "MAX_PLAYER_VALUATION" with the corresponding value from "PLAYER_VALUE"
data['MAX_PLAYER_VALUATION'] = data['MAX_PLAYER_VALUATION'].fillna(data['PLAYER_VALUE'])

## Extract player id from Transfermarkt URL using regular expressions

In [21]:
#Import regular expression library
import re
data['ID'] = data['PLAYER_HREF'].apply(lambda x: re.findall(r"/(\d+)$", x)[0])

## Drop identifier columns: players' link, club's link

In [22]:
#Select and drop columns
columns_to_drop = ['PLAYER_HREF', 'CLUB_HREF', 'UrlFBref', 'JOINED', 'MAX_PLAYER_VALUATION_DATE', 'MAX_PLAYER_VALUATION', 'LEAGUE_NAME']
data.drop(columns=columns_to_drop, inplace=True)

In [23]:
#View data shape
data.shape

(2384, 94)

In [24]:
#View columns data types
data_types_df = pd.DataFrame(data.dtypes, columns=['Data Types'])
data_types_df.to_csv("datasets/temp/TEMP_data_types.csv")

## Save cleaned dataframe

In [25]:
#Save the "data" DataFrame as a CSV file named "cleaned_dataset.csv"
data.to_csv('datasets/raw_data_final/cleaned_dataset.csv', index=False)